In [116]:
import torch
import torchvision
import numpy as np
import pandas as pd
from torch.utils.data import Dataset,DataLoader
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from torch import nn, optim
import torch.nn.functional as F
from torch.autograd import Variable
import os

In [117]:
path = os.getcwd()+'\iris.csv'

In [118]:
dataset = pd.read_csv(path)

label_encoder = preprocessing.LabelEncoder() 
  
# Encode labels in column 'species'. 
dataset['variety']= label_encoder.fit_transform(dataset['variety']) 
  
dataset['variety'].unique() 


array([0, 1, 2], dtype=int64)

In [119]:
X = dataset.drop('variety',axis=1)
y = dataset['variety']

In [120]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [121]:
trainset = pd.concat([X_train,y_train],axis=1)
testset = pd.concat([X_test,y_test],axis=1)

In [122]:
class MSRDataset(Dataset):
    def __init__(self,data):
        self.len = data.shape[0]
        self.x_data = torch.from_numpy(data.drop('variety',axis=1).to_numpy()).float()
        self.y_data = torch.from_numpy(data['variety'].to_numpy()).type(torch.LongTensor)
    
    def __getitem__(self,index):
        return self.x_data[index],self.y_data[index]
    
    def __len__(self):
        return self.len

In [123]:
train_data = MSRDataset(trainset)
test_data = MSRDataset(testset)

In [124]:
trainloader = DataLoader(dataset=train_data,batch_size=8,shuffle=True,num_workers=0)
testloader = DataLoader(dataset=test_data,batch_size=8,shuffle=True,num_workers=0)


In [125]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(4, 50)
        self.fc2 = nn.Linear(50, 20)
        self.fc3 = nn.Linear(20, 3)
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.softmax(self.fc3(x),dim=1)
        return x

In [130]:
net = Net()

In [131]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(),lr=0.001)

In [132]:
import pdb
correct = 0
total = 0
for epoch in range(100):
    for i,data in enumerate(trainloader):
        inputs,labels = data
        inputs,labels = Variable(inputs),Variable(labels)
        optimizer.zero_grad()
        output = net(inputs)
        #pdb.set_trace()
        loss = criterion(output,labels)
        loss.backward()
        optimizer.step()
    if epoch %10 == 0:
        for idx,i in enumerate(output):
            if torch.argmax(i) == labels[idx]:
                correct+=1
            total+=1
        print('Accuracy on trainset epoch',epoch,':',round(correct/total,3)*100,' loss ',loss.item())
        

Accuracy on trainset epoch 0 : 25.0  loss  1.0661256313323975
Accuracy on trainset epoch 10 : 25.0  loss  1.0670506954193115
Accuracy on trainset epoch 20 : 50.0  loss  0.7587454319000244
Accuracy on trainset epoch 30 : 62.5  loss  0.6533069014549255
Accuracy on trainset epoch 40 : 65.0  loss  0.8223469257354736
Accuracy on trainset epoch 50 : 70.8  loss  0.6673305034637451
Accuracy on trainset epoch 60 : 75.0  loss  0.5731071829795837
Accuracy on trainset epoch 70 : 78.10000000000001  loss  0.5587584376335144
Accuracy on trainset epoch 80 : 80.60000000000001  loss  0.5561586618423462
Accuracy on trainset epoch 90 : 82.5  loss  0.5539840459823608


In [133]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        inputs,labels = data
        inputs,labels = Variable(inputs),Variable(labels)
        #pdb.set_trace()
        output = net(inputs)
        for idx, i in enumerate(output):
            #pdb.set_trace()
            if torch.argmax(i) == labels[idx]:
                correct+=1
            total+=1
#
print('Accuracy ', round(correct/total,3)*100)

Accuracy  98.0
